In [1]:
from pyspark.sql import DataFrame
from pyspark.sql import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.tuning import *
from pyspark.ml.feature import Imputer

In [2]:
import numpy as np
import pandas as pd
import scipy as sc

In [4]:
dat = spark.read.csv("/hduser1/train_approx.csv",header=True, inferSchema=True)

In [5]:
test = spark.read.csv("/user/dk444/test.csv", header = True, inferSchema=True)

In [5]:
print(len(dat.columns))
print(dat.count())
dat.rdd.getNumPartitions()

114
114321


58

In [6]:
#repartition to 5 blocks
dat = dat.repartition(150)
test = dat.repartition(150)

In [7]:
dat.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- v1: double (nullable = true)
 |-- v2: double (nullable = true)
 |-- v4: double (nullable = true)
 |-- v5: double (nullable = true)
 |-- v6: double (nullable = true)
 |-- v7: double (nullable = true)
 |-- v8: double (nullable = true)
 |-- v9: double (nullable = true)
 |-- v10: double (nullable = true)
 |-- v11: double (nullable = true)
 |-- v12: double (nullable = true)
 |-- v13: double (nullable = true)
 |-- v14: double (nullable = true)
 |-- v15: double (nullable = true)
 |-- v16: double (nullable = true)
 |-- v17: double (nullable = true)
 |-- v18: double (nullable = true)
 |-- v19: double (nullable = true)
 |-- v20: double (nullable = true)
 |-- v21: double (nullable = true)
 |-- v23: double (nullable = true)
 |-- v25: double (nullable = true)
 |-- v26: double (nullable = true)
 |-- v27: double (nullable = true)
 |-- v28: double (nullable = true)
 |-- v29: double (nullable = true)
 |-- v32: double (nullable = true)
 |-- v33: double (null

In [10]:
dat = dat.drop('ID')
test = test.drop('ID')

In [8]:
# specify the number of rows to display by using the take()
dat.take(1)

[Row(_c0=12110, v1=-1.22841523304125, v2=-0.357811779916537, v4=0.817177479661627, v5=-0.811954765800889, v6=0.720647219646289, v7=-0.437880521986937, v8=-0.361606808454984, v9=0.636475765116843, v10=0.537825481827579, v11=1.79311200685637, v12=0.223156706032837, v13=-0.773794685356874, v14=-1.75694427989216, v15=-0.34562127873204, v16=-1.67263704309952, v17=-0.0195593544443429, v18=-0.876003309234192, v19=-0.107737693757444, v20=0.513535945819183, v21=-2.72304289942787, v23=-0.172529846682884, v25=-0.47995994607189, v26=-0.880319869674916, v27=-0.0313546142109312, v28=-1.15732553902148, v29=0.861397333454323, v32=-0.513475215282834, v33=-0.642982696732912, v34=2.03217103437292, v35=1.50211996142938, v36=-1.14937284152847, v37=-0.977432379413093, v38=-0.158065130020606, v39=-0.0253085080043668, v40=-1.81535143589921, v41=1.24511216910305, v42=-0.0433709071269032, v43=-1.17362253424068, v44=-0.511586986818077, v45=0.340964432465869, v46=-0.425951316058441, v48=-0.220252808885165, v49=1.

In [9]:
# how to count unique classes in a column
dat.select(col("target")).distinct().count()

2

In [10]:
# we can do an aggregate count to see the distribution of the classes
dat.groupBy(col("target")).count().collect()

[Row(target=1, count=87021), Row(target=0, count=27300)]

In [11]:
#we can do the same thing in sql code
# we must first create a "view" of the data in order to execute SQL functions
dat.createOrReplaceTempView("dat")
spark.sql("SELECT COUNT(*) FROM dat GROUP BY Target").collect()

[Row(count(1)=87021), Row(count(1)=27300)]

In [11]:
#Convert string types(categorical) to integerTypes
types = [str(f.dataType) for f in dat.schema.fields]
ind_str = [i for i in range(len(types)) if types[i] is 'StringType']
ind_int = [i for i in range(len(types)) if types[i] is 'IntegerType']
stringcol= [dat.columns[i] for i in ind_str]
intcol = [dat.columns[i] for i in ind_int]
print(stringcol)
print(intcol)
del ind_str, ind_int, types

['v3', 'v22', 'v24', 'v30', 'v31', 'v47', 'v52', 'v56', 'v66', 'v71', 'v74', 'v75', 'v79', 'v91', 'v107', 'v110', 'v112', 'v113', 'v125']
['target', 'v38', 'v62', 'v72', 'v129']


In [12]:
# remove all string columns
for colname in stringcol:
    dat = dat.drop(colname)
    test = test.drop(colname)
    
# remove 'target' from inttype list
intcol = intcol[1:]

In [13]:
intcol

['v38', 'v62', 'v72', 'v129']

In [14]:
#Imputer requires that all columns it uses be floattype or doubletype, so let's convert ints to floats
for colname in intcol:
    dat = dat.withColumn( colname+"cast", col(colname).cast("float"))
    dat = dat.drop(colname)
    test = test.withColumn( colname+"cast", col(colname).cast("float"))
    test = test.drop(colname)
    

In [15]:
#Impute missing values for both dat and test
#Let's actually impute now!
newcolnames = [colname+"new" for colname in dat.columns[1:]]
imp = Imputer(strategy = 'mean',missingValue=None, inputCols = dat.columns[1:],outputCols=newcolnames )
model = imp.fit(dat)

In [16]:
dat = model.transform(dat)
test = model.transform(test)

In [17]:
features = dat.columns[109:]
features

['v38cast',
 'v62cast',
 'v72cast',
 'v129cast',
 'v1new',
 'v2new',
 'v4new',
 'v5new',
 'v6new',
 'v7new',
 'v8new',
 'v9new',
 'v10new',
 'v11new',
 'v12new',
 'v13new',
 'v14new',
 'v15new',
 'v16new',
 'v17new',
 'v18new',
 'v19new',
 'v20new',
 'v21new',
 'v23new',
 'v25new',
 'v26new',
 'v27new',
 'v28new',
 'v29new',
 'v32new',
 'v33new',
 'v34new',
 'v35new',
 'v36new',
 'v37new',
 'v39new',
 'v40new',
 'v41new',
 'v42new',
 'v43new',
 'v44new',
 'v45new',
 'v46new',
 'v48new',
 'v49new',
 'v50new',
 'v51new',
 'v53new',
 'v54new',
 'v55new',
 'v57new',
 'v58new',
 'v59new',
 'v60new',
 'v61new',
 'v63new',
 'v64new',
 'v65new',
 'v67new',
 'v68new',
 'v69new',
 'v70new',
 'v73new',
 'v76new',
 'v77new',
 'v78new',
 'v80new',
 'v81new',
 'v82new',
 'v83new',
 'v84new',
 'v85new',
 'v86new',
 'v87new',
 'v88new',
 'v89new',
 'v90new',
 'v92new',
 'v93new',
 'v94new',
 'v95new',
 'v96new',
 'v97new',
 'v98new',
 'v99new',
 'v100new',
 'v101new',
 'v102new',
 'v103new',
 'v104new

In [162]:
# convert the columns into a single feature vector

assembler = VectorAssembler(
    inputCols=[x for x in features],
    outputCol='features')

dat_processed = assembler.transform(dat)
test_processed = assembler.transform(test)

In [166]:
dat_processed.printSchema()

root
 |-- target: integer (nullable = true)
 |-- v1: double (nullable = true)
 |-- v2: double (nullable = true)
 |-- v4: double (nullable = true)
 |-- v5: double (nullable = true)
 |-- v6: double (nullable = true)
 |-- v7: double (nullable = true)
 |-- v8: double (nullable = true)
 |-- v9: double (nullable = true)
 |-- v10: double (nullable = true)
 |-- v11: double (nullable = true)
 |-- v12: double (nullable = true)
 |-- v13: double (nullable = true)
 |-- v14: double (nullable = true)
 |-- v15: double (nullable = true)
 |-- v16: double (nullable = true)
 |-- v17: double (nullable = true)
 |-- v18: double (nullable = true)
 |-- v19: double (nullable = true)
 |-- v20: double (nullable = true)
 |-- v21: double (nullable = true)
 |-- v23: double (nullable = true)
 |-- v25: double (nullable = true)
 |-- v26: double (nullable = true)
 |-- v27: double (nullable = true)
 |-- v28: double (nullable = true)
 |-- v29: double (nullable = true)
 |-- v32: double (nullable = true)
 |-- v33: double (n

In [13]:
# Create a logistic regression model, tree model, and a gradient boosted tree model
# Create an evaluator object

logit = LogisticRegression(featuresCol="features",labelCol="Target")
logit_model = logit.fit(train)

cart = DecisionTreeClassifier(maxDepth=10)
cart_model = cart.fit(train)

gbt = GBTClassifier(maxDepth=10, stepSize=.001,maxIter=10)
gbt_model = gbt.fit(train)

In [14]:
evaluator = BinaryClassificationEvaluator(labelCol='label')

In [15]:
pred_logit = logit_model.transform(test)
pred_cart = cart_model.transform(test)
pred_gbt = gbt_model.transform(test)

In [16]:
results = [evaluator.evaluate(i) for i in [pred_logit, pred_cart, pred_gbt]]

In [17]:
print(results)

[0.985497295510151, 0.8885200864455169, 0.8875588161461015]


In [19]:
56897/(56897+106)

0.9981404487483115

In [21]:
col_count = len(dat.columns)

assembler = VectorAssembler(
    inputCols=[x for x in features],
    outputCol='features')

dat_processed = assembler.transform(dat)
test_processed = assembler.transform(test)

normalizer = Normalizer(inputCol="features",outputCol="feature")

dat = normalizer.transform(assembler.transform(dat))

In [22]:
# Concept of pipeline
# Additionally how to cross validate
logit = LogisticRegression(featuresCol='feature',labelCol='target')
cart = DecisionTreeClassifier(featuresCol='feature',labelCol='target')
gbt = GBTClassifier(featuresCol='feature',labelCol='target')


paramGrid_logit = ParamGridBuilder() \
    .addGrid(logit.regParam, [0,0.01, 0.1]) \
    .build()
    
paramGrid_cart = ParamGridBuilder() \
    .addGrid(cart.maxDepth, [10,12,15]) \
    .build()
    
paramGrid_gbt = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [10,12,15]) \
    .addGrid(gbt.stepSize,[0.01]) \
    .addGrid(gbt.maxIter,[20]) \
    .build()

evaluator = BinaryClassificationEvaluator(labelCol='target')
    

cv_logit = CrossValidator(estimator=logit,evaluator=evaluator,estimatorParamMaps=paramGrid_logit,numFolds=10)
cv_cart = CrossValidator(estimator=cart,evaluator=evaluator,estimatorParamMaps=paramGrid_cart,numFolds=10)
cv_gbt = CrossValidator(estimator=gbt,evaluator=evaluator,estimatorParamMaps=paramGrid_gbt,numFolds=10)


In [24]:
cvmodel_logit = cv_logit.fit(dat)

In [ ]:
cvmodel_cart = cv_cart.fit(dat)

In [ ]:
cvmodel_gbt = cv_gbt.fit(dat)

In [169]:
print(cvmodel_logit.avgMetrics)
print(cvmodel_cart.avgMetrics)
print(cvmodel_gbt.avgMetrics)

[0.7087400464393288, 0.6992343001874801, 0.7080240843897794]
[0.5036192095312046, 0.5331364209963431, 0.5430835878468885, 0.5739519392919172]
[0.7251348350063479, 0.7448645684366266, 0.7684780371882793, 0.8201436144971819]


In [43]:
np.argmax(np.array(cvmodel_gbt.avgMetrics))

3

In [ ]:
rf = RandomForestClassifier(featuresCol='feature',labelCol="target")
pipeline_rf = Pipeline(stages=[assembler,normalizer, rf])

paramGrid_rf = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [15]) \
    .addGrid(rf.numTrees, [1000]) \
    .build()
    
cv_rf = CrossValidator(estimator=pipeline_rf, evaluator=evaluator, numFolds=10, estimatorParamMaps=paramGrid_rf)
cvmodel_rf = cv_rf.fit(dat)
cvmodel_rf.avgMetrics

In [58]:
dat.rdd.sampleByKey(withReplacement=False,fractions={0:0.0001, 1:1})

In [61]:
dat.rdd.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 178673.0 failed 4 times, most recent failure: Lost task 0.3 in stage 178673.0 (TID 556659, mylaptop, executor 3): java.io.IOException: Cannot run program "/home/hduser1/anaconda3/bin/python": error=2, No such file or directory
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:163)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:89)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:65)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.PythonRunner.compute(PythonRDD.scala:128)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:63)
	at org.apache.spark.rdd.RDD.comp